In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../src")

In [8]:
from data import MNISTDatamodule
from model import ImageClassifier
from torch import nn
from torchvision import transforms as T
import pytorch_lightning as pl
from architectures.feature_extractors.resnet import ResNet

In [9]:
# mean = [0.485, 0.456, 0.406]
# std = [0.229, 0.224, 0.225]
mean = [0.1307]
std = [0.3081]
transform = T.Compose(
    [
        T.ToTensor(),
        T.Normalize(mean=mean, std=std),
        # nn.Flatten(start_dim=0, end_dim=-1)
    ]
)

In [10]:
dm = MNISTDatamodule(
    data_dir="../data",
    transform = transform,
)

In [11]:
dm.download_data()
dm.setup()

In [12]:
X, y = next(iter(dm.train_dataloader()))

In [13]:
from torchinfo import summary
import torch

In [34]:
resnet = ResNet(
    in_channels=1,
    stem_channels=64,
    stem_kernel_size=3,
    block_type="basic",
    stages_n_blocks=[1, 3],
    pool_kernel_size=1
)

In [35]:
resnet(X).shape

torch.Size([64, 128])

In [36]:
resnet.out_shape

128

In [23]:
64 * (2**(3-1))

256